## Summarization Inference using Online Endpoints

This sample shows how to deploy `summarization` type models to an online endpoint for inference.

### Task
`summarization` creates a shorter version of a document or an article that captures all the important information. Along with translation, it is another example of a task that can be formulated as a sequence-to-sequence task. 
`summarization` can be:

* Extractive: extract the most relevant information from a document.
* Abstractive: generate new text that captures the most relevant information.

### Model
Models that can perform the `summarization` task are tagged with `task: summarization`. We will use the `sshleifer-distilbart-cnn-12-6` model in this notebook. If you opened this notebook from a specific model card, remember to replace the specific model name. If you don't find a model that suits your scenario or domain, you can discover and [import models from HuggingFace hub](../../import/import-model-from-huggingface.ipynb) and then use them for inference. 

### Inference data
We will use the [CNN DailyMail](https://huggingface.co/datasets/cnn_dailymail) dataset. A copy of this dataset is available in the [news-summary-dataset](./news-summary-dataset/) folder.

### Outline
* Set up pre-requisites.
* Pick a model to deploy.
* Prepare data for inference. 
* Deploy the model for real time inference.
* Test the endpoint
* Clean up resources.

### 1. Set up pre-requisites
* Install dependencies
* Connect to AzureML Workspace. Learn more at [set up SDK authentication](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-setup-authentication?tabs=sdk). Replace  `<WORKSPACE_NAME>`, `<RESOURCE_GROUP>` and `<SUBSCRIPTION_ID>` below.
* Connect to `azureml` system registry

In [1]:
from azure.ai.ml import MLClient
from azure.identity import (
    DefaultAzureCredential,
    InteractiveBrowserCredential,
    ClientSecretCredential,
)
from azure.ai.ml.entities import AmlCompute
import time

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()

workspace_ml_client = MLClient(
    credential,
    subscription_id="ea4faa5b-5e44-4236-91f6-5483d5b17d14",
    resource_group_name="amyharrispersonal",
    workspace_name="amyharris-canary",
)
# the models, fine tuning pipelines and environments are available in the AzureML system registry, "azureml-preview"
registry_ml_client = MLClient(credential, registry_name="azureml-preview")

Class FeatureStoreOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class FeatureSetOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class FeatureStoreEntityOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


### 2. Pick a model to deploy

Browse models in the Model Catalog in the AzureML Studio, filtering by the `summarization` task. In this example, we use the `sshleifer-distilbart-cnn-12-6` model. If you have opened this notebook for a different model, replace the model name and version accordingly. 

In [2]:
model_name = "sshleifer-distilbart-cnn-12-6"
model_version = "3"
foundation_model = registry_ml_client.models.get(model_name, model_version)
print(
    "\n\nUsing model name: {0}, version: {1}, id: {2} for inferencing".format(
        foundation_model.name, foundation_model.version, foundation_model.id
    )
)



Using model name: sshleifer-distilbart-cnn-12-6, version: 3, id: azureml://registries/azureml-preview/models/sshleifer-distilbart-cnn-12-6/versions/3 for inferencing


### 3. Prepare data for inference.

A subset of the news summary dataset is available in the [news-summary-dataset](./news-summary-dataset/) folder. The next few cells show basic data preparation:
* Visualize some data rows
* Save few samples in the format that can be passed as input to the online-inference endpoint.

In [3]:
# load the ./news-summary-dataset/train_100.jsonl file into a pandas dataframe and show the first 5 rows
import pandas as pd

pd.set_option(
    "display.max_colwidth", 0
)  # set the max column width to 0 to display the full text
train_df = pd.read_json("./news-summary-dataset/train_100.jsonl", lines=True)
train_df.head(2)

,article,highlights,id
0,"MADRID, Spain (CNN) -- Barack Obama sometimes gets ribbed for his outsized ideas, like the massive stage built in Denver, Colorado, to accept the Democratic presidential nomination. But an artist in Barcelona, Spain, may be about to outdo the candidate himself. Large-format sketches have long been a focus of artist Jorge Rodriguez-Gerada's work. The artist plans to create a gigantic face of Obama sculpted from gravel and sand, which will cover nearly 2.5 acres (1 hectare) of Barcelona beachfront before the U.S. elections. ""The size of the piece is intrinsic to its value,"" the artist, Jorge Rodriguez-Gerada, said Saturday. He hopes it will be big enough to be seen on Google Earth. ""Obama's personality -- his youth, personal history and message of a new politics -- has fused with the historical moment to create someone larger than life,"" says the artist's dossier about the work, titled ""Expectation."" The huge size also alludes ""to the global impact of this election,"" the dossier adds. iReport.com: What do you want to ask Sen. Obama? Rodriguez-Gerada, 42, is a Cuban-born American who grew up in North Plainfield, New Jersey, near New York, and now lives in Barcelona. The artist, who has long focused on large-format sketches and other designs, said he had been planning to put an anonymous face on Barcelona's Mediterranean beachfront but shelved the idea. Then, six weeks ago, he decided it should be Obama's face. ""Everybody's doing work about Obama,"" he said. ""I was talking to my wife about the importance of this election internationally. It all came together."" The idea, that is. The execution of the project is still a work in progress, but with a lot of help pouring in. The artist has created what he calls a ""vectored image"" from different photos of Obama, showing the candidate's face looking left over his shoulder. He will use 500 tons of material -- mainly gravel but also sand and possibly some soil in black, brown and white tones -- to create the image on a flat piece of land slightly raised and overlooking the Mediterranean, near downtown Barcelona. The artist and a Barcelona newspaper say the city has temporarily ceded use of the land for the project. The portrait of Obama's face will be 445 feet long by 264 feet wide (139.28 by 82.67 meters) and the artist hopes to have it done by late next week or no later than November 3, the day before the U.S. elections. The materials have been donated, along with bulldozers and their drivers. The crews will initially spread the materials along white lines, and other markers that will be laid out according to the sketch. Then, the artist's technical team will direct volunteers with garden rakes to put the final touches on the materials, making sure the various colors are spread properly on the ground in order to depict Obama's eyes, hair, cheeks and collar, according to the plans. Rodriguez-Gerada said he'll need to raise about $18,700 (15,000 euros) for other costs, such as rakes and gloves for volunteers, documentation, even portable toilets for the crew. He said it's his ""biggest work ever, in scale and complexity."" He's a self-described political independent until recently, when he joined Democrats Abroad in Spain. But he added that the art project is his alone, without help from any political organization or campaign, and he won't be paid for it. Rodriguez-Gerada said this project also aims to address the issue of ""trying to find heroes with empathy for the problems of the world. We really need empathy from Barack Obama. There's a need to do positive things for the future."" Obama's giant face would be environmentally friendly, with all natural colors, so the materials could be recycled, the artist said. Rodriguez-Gerada's not sure how long the face would remain in place in Barcelona, because the site is slated to be used for a new municipal building. But even if the face isn't permanent, it might be the start of outsized artistic images of 

### 4. Deploy the model to an online endpoint
Online endpoints give a durable REST API that can be used to integrate with applications that need to use the model.

In [4]:
import time, sys
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    OnlineRequestSettings,
)

# Create online endpoint - endpoint names need to be unique in a region, hence using timestamp to create unique endpoint name
timestamp = int(time.time())
online_endpoint_name = "summarization-" + str(timestamp)
# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Online endpoint for "
    + foundation_model.name
    + ", for summarization task",
    auth_mode="key",
)
workspace_ml_client.begin_create_or_update(endpoint).wait()

In [5]:
# create a deployment
demo_deployment = ManagedOnlineDeployment(
    name="demo",
    endpoint_name=online_endpoint_name,
    model=foundation_model.id,
    instance_type="Standard_DS3_v2",
    instance_count=1,
    request_settings=OnlineRequestSettings(
        request_timeout_ms=60000,
    ),
)
workspace_ml_client.online_deployments.begin_create_or_update(demo_deployment).wait()
endpoint.traffic = {"demo": 100}
workspace_ml_client.begin_create_or_update(endpoint).result()

Check: endpoint summarization-1684194623 exists
data_collector is not a known attribute of class <class 'azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.ManagedOnlineDeployment'> and will be ignored


..........................................................................................................................................................................................

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://summarization-1684194623.eastus2euap.inference.ml.azure.com/score', 'openapi_uri': 'https://summarization-1684194623.eastus2euap.inference.ml.azure.com/swagger.json', 'name': 'summarization-1684194623', 'description': 'Online endpoint for sshleifer-distilbart-cnn-12-6, for summarization task', 'tags': {}, 'properties': {'azureml.onlineendpointid': '/subscriptions/ea4faa5b-5e44-4236-91f6-5483d5b17d14/resourcegroups/amyharrispersonal/providers/microsoft.machinelearningservices/workspaces/amyharris-canary/onlineendpoints/summarization-1684194623', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/ea4faa5b-5e44-4236-91f6-5483d5b17d14/providers/Microsoft.MachineLearningServices/locations/eastus2euap/mfeOperationsStatus/oe:c76e6446-545b-4141-80f9-e8ad59c471f2:d50be4cc-d54d-4627-8c85-9b4e8f7a4747?api-version=2022-02-01-preview'}, 'print_as_yaml': True, 'id'

### 5. Test the endpoint with sample data

We will fetch some sample data from the test dataset and submit to online endpoint for inference. We will then show the display the scored labels alongside the ground truth labels

In [6]:
import json
import os

# read the ./news-summary-dataset/train_100.jsonl file into a pandas dataframe
df = pd.read_json("./news-summary-dataset/train_100.jsonl", lines=True)
# escape single and double quotes in the masked_text column
df["article"] = df["article"].str.replace("'", "\\'").str.replace('"', '\\"')
# pick 1 random row
sample_df = df.sample(1)
# create a json object with the key as "inputs" and value as a list of values from the article column of the sample_df dataframe
sample_json = {"inputs": sample_df["article"].tolist()}
# save the json object to a file named sample_score.json in the ./news-summary-dataset folder
test_json = {"inputs": {"input_string": sample_df["article"].tolist()}}
# save the json object to a file named sample_score.json in the ./news-summary-dataset folder
with open(os.path.join(".", "news-summary-dataset", "sample_score.json"), "w") as f:
    json.dump(test_json, f)
sample_df.head()

,article,highlights,id
85,"(CNN) -- Kate Hudson\'s ex, Black Crowes rocker Chris Robinson, is going to be a dad again, a representative for the band confirmed in a statement Tuesday. Chris Robinson and girlfriend Allison Bridges will be having a child in early 2010. Robinson and girlfriend Allison Bridges, who have been dating for two years, are expecting their first child in early 2010, the statement said. The baby will be the 42-year-old frontman\'s second child --­ he and Hudson have a 5 1/2-year-old son, Ryder Russell, together. Hudson and Robinson were married for six years and their divorce was finalized in October 2006. They were granted joint custody of their son. Robinson and his brother Rich formed the band that would eventually become the Black Crowes in the 1980s. The Crowes\' new album, \""Before the Frost . . . Until the Freeze,\"" is in stores now.","Black Crowes frontman Chris Robinson having baby with girlfriend .\nRobinson and girlfriend Allison Bridges have been dating for two years .\nHe and ex Kate Hudson have a 5-1/2-year-old son, Ryder Russell, together .",c6eaf9d97b059f3e824a1ab4ffdfe45494e5f8a1


In [7]:
# score the sample_score.json file using the online endpoint with the azureml endpoint invoke method
response = workspace_ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="demo",
    request_file="./news-summary-dataset/sample_score.json",
)
print("raw response: \n", response, "\n")
# convert the json response to a pandas dataframe
response_df = pd.read_json(response)
response_df.head()

raw response: 
 [{"0": " Chris Robinson and girlfriend Allison Bridges are expecting their first child in early 2010 . The baby will be the 42-year-old frontman\\'s second child --\u00ad he and Kate Hudson have a 5 1/2 year-old son, Ryder Russell . Hudson and Robinson were married for six years and their divorce was finalized in October 2006 ."}] 



,0
0,"Chris Robinson and girlfriend Allison Bridges are expecting their first child in early 2010 . The baby will be the 42-year-old frontman\'s second child --­ he and Kate Hudson have a 5 1/2 year-old son, Ryder Russell . Hudson and Robinson were married for six years and their divorce was finalized in October 2006 ."


In [8]:
# compare the true summary with the predicted summary
response_df.rename(columns={"summary_text": "predicted_summary"}, inplace=True)
response_df["ground_truth_summary"] = [sample_df["highlights"].tolist()[0]]
response_df.head()

,0,ground_truth_summary
0,"Chris Robinson and girlfriend Allison Bridges are expecting their first child in early 2010 . The baby will be the 42-year-old frontman\'s second child --­ he and Kate Hudson have a 5 1/2 year-old son, Ryder Russell . Hudson and Robinson were married for six years and their divorce was finalized in October 2006 .","Black Crowes frontman Chris Robinson having baby with girlfriend .\nRobinson and girlfriend Allison Bridges have been dating for two years .\nHe and ex Kate Hudson have a 5-1/2-year-old son, Ryder Russell, together ."


### 6. Delete the online endpoint
Don't forget to delete the online endpoint, else you will leave the billing meter running for the compute used by the endpoint

In [9]:
workspace_ml_client.online_endpoints.begin_delete(name=online_endpoint_name).wait()

............